In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from utils.PerformanceMetrics import RegressionEvaluation
from utils.create_dataset import create_dataset

In [ ]:
class Parameters():
    def __init__(self):
        # Model parameters
        # ----------------------------------------------------------------
        # Model description
        self.description = 'Convolutional-based Kolmogorov-Arnold network model for time-series forecasting'     
        # Input sequence length - look-back
        self.Lag = 12
        # Prediction sequence length
        self.Horizon = 4
        # Parameters
        self.Conv_parameters = {'out_channels': ..., 'kernel_size': ..., 'stride': ..., 'padding': ...}
        self.steps = ...
        self.width = [6, 3, self.Horizon]
        self.grid = ...
        self.k=...
        self.lr=...
        self.lamb=...
        self.lamb_l1=...
        self.lamb_entropy=...
        # Device {'cpu', 'cuda:0'}
        self.device = "cuda:0"

args = Parameters()

### Data

In [ ]:
# Start timer
start = time.time()

# Load data
df = pd.read_csv("data/BTC-USD.csv")[["Date", "Close"]]
# Pre-process data (Log-series to reduce volatility)
df["Close"] = np.log(df["Close"])
# Convert Date to 'datetime64'
df['Date'] = df['Date'].astype('datetime64[ns]')
# Set index
df.set_index('Date', inplace=True)
print('[INFO] Data imported')
print('[INFO] Time: %.2f seconds' % (time.time() - start))

# Split Train/Test
idx = int( df.shape[0] * 0.8 )
df_train = df[:idx].dropna()
df_test  = df[idx:].dropna()

# Visualization
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(20, 3) )
df_train["Close"].plot(ax=ax, color='tab:blue' )
df_test["Close"].plot(ax=ax,  color='tab:orange')
plt.legend(['Training', 'Testing'], frameon = False, fontsize = 14)
plt.ylabel("Road_occupancy_rates", size = 14)
plt.xlabel('Date', size = 14);
plt.xticks(size = 12);
plt.yticks(size = 12);

In [ ]:
trainX, trainY, _ = create_dataset(df = df_train, 
                                   Lag = args.Lag, 
                                   Horizon = args.Horizon, 
                                   overlap = 1)
                               
testX,  testY, testDate  = create_dataset(df = df_test, 
                                          Lag = args.Lag, 
                                          Horizon = args.Horizon, 
                                          overlap = args.Horizon)

# Resizing
trainY = trainY.squeeze(-1)
testY = testY.squeeze(-1)
trainX = trainX.transpose(0, 2, 1)
testX = testX.transpose(0, 2, 1)

# Last 10% of the training data will be used for validation
#
idx = int(0.8 * trainX.shape[0])
validX, validY = trainX[idx:], trainY[idx:]
trainX, trainY = trainX[:idx], trainY[:idx]



print('Training data shape:   ', trainX.shape, trainY.shape)
print('Validation data shape: ', validX.shape, validY.shape)
print('Testing data shape:    ', testX.shape,  testY.shape)
    
dataset = dict()
dataset['train_input'] = torch.from_numpy(trainX).float().to(args.device)
dataset['test_input'] = torch.from_numpy(testX).float().to(args.device)
dataset['train_label'] = torch.from_numpy(trainY).float().to(args.device)
dataset['test_label'] = torch.from_numpy(testY).float().to(args.device)

### Model

In [ ]:
from kan import CKAN
from utils.dilate_loss import dilate_loss
def loss_fn(x, y):
    loss, _, _ = dilate_loss(x.unsqueeze(-1), y.unsqueeze(-1), alpha=0.9, gamma=0.1, device=args.device)
    return loss

model = CKAN(Lag=args.Lag, Conv_parameters=args.Conv_parameters, width=args.width, grid=args.grid, k=args.k, device=args.device)
results = model.train(dataset, opt="LBFGS", lr=args.lr, steps=args.steps, lamb=args.lamb, lamb_l1=args.lamb_l1, lamb_entropy=args.lamb_entropy, device=args.device, save_fig=False, loss_fn=loss_fn);

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols = 3, figsize=(15, 2))

# Train & Test Losses
ax[0].plot(results['train_loss'], marker="o", label="Train loss")
ax[0].plot(results['test_loss'], marker="o", label="Test loss")
ax[0].legend(frameon=False);

# RMSE (test)
ax[1].plot(results['test_RMSE'], marker="o", color='g', label="RMSE (test)")
ax[1].legend(frameon=False);

# R2 (test)
ax[2].plot(results['test_R2'], marker="o", color='g', label="R2 (test)")
ax[2].legend(frameon=False);

### Evaluation

In [ ]:
# Get predictions
pred = model(torch.tensor(testX).float()).cpu().detach().numpy()

# Inverse-Transformation
testY, pred = np.exp(testY), np.exp(pred)

# Evaluation 
MAE, RMSE, MAPE, SMAPE, Hausdorff_score, ramp_score, R2 = RegressionEvaluation(testY.flatten(), pred.flatten())
print('MAE %5.2f | RMSE %5.2f | SMAPE: %5.2f | Hausdorff_score: %5.2f | Ramp_score: %5.2f | R2: %.2f' % (MAE, RMSE, SMAPE, Hausdorff_score, ramp_score, R2) )